# INFO F422 - Statistical Fundation of Machine Learning
## Project "House Prices : Advanced Regression Techniques"

    Erica Berghman
    Master 1 - Brussels Engineer School

### Abstract

### Introduction 
% with dataset description, goals, and an overview of the report structure 
Starting from a data set with xx criteria about houses and their selling price, the goal is to create a model capable of predicting the price of other houses given some of these criterias. A good model description is a model that has been refined multiple types. This report will show the methodology used to construct a model for this particular problem. It is based on the methodology of the Chapter 6 of the syllabus.

### Feature selection 
Methodology and main results

The text must contain the list of selected variables and the motivation of their choice. The use of formulas, tables and pseudo-code to describe the feature selection procedure is encouraged. 

In [27]:
dataSample = 400
mean = T # variable to determine if we use the mean or the median to replace the NA values

source("replaceNA.R")

#### Preprocessing the data set

In order to get a model, the data must be preprocessed. Firstly we read the data given and we take a sample set of 400 houses out of the 1460. There is 81 criteria.

In [28]:
data<-read.csv("input/train.csv")
set.seed(2)
data.sample<-data[sample(nrow(data),400),]
dim(data.sample)
data[1:2,]

[1] 400  81

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1,60,RL,65,8450,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,2,2008,WD,Normal,208500
2,20,RL,80,9600,Pave,NA,Reg,Lvl,AllPub,...,0,NA,NA,NA,0,5,2007,WD,Normal,181500


The "Id" column which does not give any information is deleted. The categorical (factor) criterias are removed.

In [29]:
data.sample<-data.sample[,-1]

factor_variables<-which(sapply(data.sample[1,],class)=="factor")
data.sample$nofactor<-data.sample[,-factor_variables]
data.sample$factor<-data.sample[,factor_variables]

The missing values (NA) are replaced by an estimator of these values (eg. mean or median).

In [31]:
if (mean) {
    data_preprocessed<-data.frame(apply(data.sample$nofactor,2,replace_na_with_mean_value)) 
} else {
    data_preprocessed<-data.frame(apply(data.sample$nofactor,2,replace_na_with_median_value))
}

Once this is done, the next step is to delete the criterias that are redundant, i.e. the ones thare are linear combination of others.

In [32]:
library(caret)
findLinearCombos(data_preprocessed)

$linearCombos
$linearCombos[[1]]
[1] 12  9 10 11

$linearCombos[[2]]
[1] 16 13 14 15


$remove
[1] 12 16

In [33]:
data_preprocessed<-data_preprocessed[,-c(12,16)]  #delete combili
findLinearCombos(data_preprocessed)  # no more combili

$linearCombos
list()

$remove
NULL


### Model selection  
Methodology and main results

For the learning method, the only packages that may be used are those seen during the exercise classes : stats, nnet, tree, lazy, and e1071, for linear models, neural networks, decision trees, nearest neighbours and SVM, respectively.

The accuracy of the regression models during the selection process should be assessed by using the root mean squared error between the logarithm of the
predicted value and the logarithm of the observed sale price.

The text must mention the different
(and at least three) models which have been taken into consideration and the procedure used for model assessment and selection. The use of formulas,
tables and pseudo-code to describe the feature selection procedure is encouraged

### Ensemble techniques : Combination of models strategy
Methodology and main results

The text should mention the different models taken into consideration as well as the techniques used for the combination.

### Discussion and conclusion: 
Summary of your work, and discussion of what worked well, not well, why, what insights you got from the analyses you made. 